1. Backpropagation 구현해보기

In [ ]:
class MulLayer:
    
    def __init__(self):
        #곱셈을 할땐 input값이 2개이므로 x,y를 초기화 해주고, 무엇을 곱할지 모르므로 None처리 해둔다.
        self.x = None
        self.y = None
    
    #곱셈 계층의 forward를 표현
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    #곱셈 계층의 backward를 표현
    def backward(self,dout):
        dx = dout*self.y
        dy = dout*self.x
        
        return dx,dy
    
class AddLayer:
    
    #덧셈노드는 그냥 넘겨주기만 하면 되므로 초기화해줄 변수가 딱히 없다.
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out = x + y
        return out
    
    def backward(self,dout):
        
        dx = dout*1
        dy = dout*1
        
        return dx,dy

사과 쇼핑 예제

In [ ]:
#사과1개 - 100원 / 2개 구매 / 부가세 10%
apple = 100
apple_num = 2
tax = 1.1

#계층만들기
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print("Price : ",price)

#역전파
dprice = 1 #언제나 시작은 1 (자기 자신을 미분하면 1이기 때문)
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple,dapple_price = mul_apple_layer.backward(dapple_price)
print("dapple,dapple_price,dtax : ",dapple,",",dapple_price,",",dtax)
print("-"*100)

#사과2개와 귤3개를 사는 계산 그래프를 작성
apple = 100
orange = 150
apple_num = 2
orange_num = 3
tax = 1.1

#계층 만들기
mul_apple_layer = MulLayer()
mul_orange_layer= MulLayer()
add_fruit_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
fruit_price = add_fruit_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(fruit_price,tax)

print("price : ",price)

#역전파
dprice = 1
dfruit_price,dtax = mul_tax_layer.backward(dprice)
dapple_price,dorange_price = add_fruit_layer.backward(dfruit_price)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
dorange,dorange_num = mul_orange_layer.backward(dorange_price)
print("dapple,dorange,dapple_num,dorange_num,dtax : ",dapple,round(dorange,2),round(dapple_num,2),dorange_num,dtax)

활성화 함수 계층 구현하기 - ReLU / Softmax

In [1]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self,x):
        self.mask = (x<=0) #True/False로 이루어진 numpy array
        out = x.copy()
        out[self.mask] = 0 #(0보다 작으면 True이므로 0보다 작은값들은 0으로 채워짐)
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

class Sigmoid:
    
    def __init__(self):
        self.out = out
        
    def forward(self,x):
        out = 1/(1+np.exp(-x))
        return out
    
    def backward(self,dout):
        
        dx = dout*self.out*(1-self.out)
        
        return dx

(batch용 - 데이터를 N개 묶어서 계산한다는 뜻) Affine / softmax 계층 구현하기

In [12]:
import numpy as np

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None #입력받아봐야 알음.
        self.dw = None #가중치 업데이트를 위한 저장
        self.db = None #편향 업데이트를 위한 저장
        #self.dx ??
        
    def forward(self,x):
        #입력값 저장
        self.x = x
        out = np.dot(x,self.W) + self.b
        
        return out
    
    def backward(self,dout):
        #곱셈 계층이므로 반대(행렬의 크기로 인한 곱셈 위치 주의!)
        dx = np.dot(dout,self.W.T)
        self.dw = np.dot(self.x.T,dout)
        #크기를 맞추기 위해(?) axis = 0 으로 더하기.
        self.db = np.sum(dout,axis = 0)
        
        return dx

#softmax = np.exp()/np.sum(np.exp()) (중간 생략)
class softmax_with_loss:
    
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    #######import하기 귀찮아서 그냥 가져옴  #############
    def softmax(self,x):
        c = np.max(x)
        exp_x = np.exp(x-c)
        sum_a = np.sum(exp_x)
        return exp_x/sum_a
    
    def cross_entropy_error(self,y,t):
        if y.ndim == 1:
            y = y.reshape(1,y.size)
            t = t.reshape(1,t.size)
        batch_size = y.shape[0]
        
        return -np.sum(t*np.log(y+1e-7)) /  batch_size # t = 1 이므로 생략
    #####################################################
    
    def forward(self,x,t):
        self.t = t
        #input데이터로 softmax를 계산
        self.y = self.softmax(x)
        #최종적으로 정답과 비교하여 loss를 계산(이때 사용하는 함수는 cross entropy error 함수를 사용.)
        self.loss = self.cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout= 1):
        #한번에 몇개를 훈련시키는지 파악한다
        batch_size = self.t.shape[0]
        
        dx = (self.y - self.t)/batch_size #왜 batch_size로 나눌까? --> 데이터1개당 오차를 보내기 위해?
        
        return dx
    


위에서 구현한 클래스를 기반으로 신경망 구현해보기  
1. 미니배치
훈련 데이터중에 일부분을 무작위로 가져온다.(실제로는 random shuffle하고 일정 크기로 쪼개어서 순차대로 가져옴)  
-->신경망 학습의 목적은 이렇게 뽑아온 미니배치의 손실함수를 줄이는것이다.
2. 기울기 산출(Gradient descent optimization)
-->미니배치의 loss를 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.  
   기울기는 손실함수의 값을 가장 작게 만드는 방향을 제시해주는 역할
3. 매개변수 갱신
--> 가중치 매개변수를 기울기 방향으로 아주 조금(learning_rate(보통은 0.1) * dL/dx) 갱신한다.
4. 반복
위의 작업을 반복하여 loss를 계속 줄여나간다.


In [14]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict

class TwoLayerNet2:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std = 0.01):
        
        #가중치 초기화
        self.params = {}
        self.params["W1"] = weight_init_std*np.random.randn(input_size,hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std*np.random.randn(hidden_size,output_size)
        self.params["b2"] = np.zeros(output_size)
        
        #계층생성
        self.layers = OrderedDict() #왜 계층을 OrderedDict로 구현할까? --> 순서대로 실행하기 때문
        self.layers["Affine1"] = Affine(self.params["W1"],self.params["b1"])
        self.layers["ReLU1"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W2"],self.params["b2"])
        self.lastLayer = softmax_with_loss()
    
    
    #predict
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    #loss
    def loss(self,x,t):
        #마지막 계층전까지 predict함수를 사용하여 학습을 진행한다.
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    #accuracy
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis = 1)
        if t.ndim != 1 : t = np.argmax(t,axis = 1)
        
        accuracy = np.sum(y == t)/float(x.shape[0])
        return accuracy
    
    def numerical_gradient_V1(self,f,x):
        h = 1e-4
        grad = np.zeros_like(x)
        
        for idx in range(x.size):
            
            tem_val = x[idx]
            
            #f(x+h)
            x[idx] = tem_val + h
            fxh1 = f(x)
            
            #f(x-h)
            x[idx] = tem_val - h
            fxh2 = f(x)
            
            x[idx] = tem_val
            
            grad[idx] = (fxh1 - fxh2) / (2*h)
            
        return grad
    
    def numerical_gradient_V2(self,f,x):
        
        if x.ndim == 1 : 
            return self.numerical_gradient_V1(f,x)
        else:
            grad = np.zeros_like(x)
            for idx,point in enumerate(x):
                grad[idx] = self.numerical_gradient_V1(f,point)
        return grad
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W : self.loss(x,t)
        
        grads = {}
        grads["W1"] = self.numerical_gradient_V2(loss_W,self.params["W1"])
        grads['b1'] = self.numerical_gradient_V2(loss_W,self.params["b1"])
        grads["W2"] = self.numerical_gradient_V2(loss_W,self.params["W2"])
        grads["b2"] = self.numerical_gradient_V2(loss_W,self.params["b2"])
        
        return grads
    
    def gradient(self,x,t):
        
        #순전파
        self.loss(x,t)
        
        #역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        #결과저장
        grads = {}
        grads["W1"] = self.layers["Affine1"].dw
        grads["b1"] = self.layers["Affine1"].db
        grads["W2"] = self.layers["Affine2"].dw
        grads["b2"] = self.layers["Affine2"].db
        
        return grads
    
if __name__ == '__main__':
    import numpy as np
    from dataset.mnist import load_mnist
    
    (x_train,t_train),(x_test,t_test) = load_mnist(normalize = True,one_hot_label = True)
    train_loss_list = []
    
    #hyperparameter
    iter_num = 100
    train_size = x_train.shape[0]
    batch_size = 100
    learning_rate = 0.1
    
    #Model
    net = TwoLayerNet2(input_size = 784,hidden_size = 50,output_size = 10)
    
    for i in range(iter_num):
        #미니 배치
        batch_mask = np.random.choice(train_size,batch_size)
        x_batch = x_train[batch_mask]
        t_batch = t_train[batch_mask]
        #기울기 계산
        grad = net.gradient(x_batch,t_batch)
        
        #매개변수 갱신
        for key in ("W1","b1","W2","b2"):
            net.params[key] -= learning_rate*grad[key]
            
        #학습 경과 기록
        loss = net.loss(x_batch,t_batch)
        train_loss_list.append(loss)
        print(i,'회차')
        print("loss : ",round(loss,3))

0 회차
loss :  6.906
1 회차
loss :  6.905
2 회차
loss :  6.905
3 회차
loss :  6.906
4 회차
loss :  6.904
5 회차
loss :  6.903
6 회차
loss :  6.901
7 회차
loss :  6.899
8 회차
loss :  6.898
9 회차
loss :  6.902
10 회차
loss :  6.895
11 회차
loss :  6.894
12 회차
loss :  6.888
13 회차
loss :  6.893
14 회차
loss :  6.901
15 회차
loss :  6.905
16 회차
loss :  6.931
17 회차
loss :  6.934
18 회차
loss :  7.018
19 회차
loss :  7.163
20 회차
loss :  7.679
21 회차
loss :  7.927
22 회차
loss :  9.236
23 회차
loss :  10.61
24 회차
loss :  12.519
25 회차
loss :  14.571
26 회차
loss :  14.754
27 회차
loss :  15.934
28 회차
loss :  15.956
29 회차
loss :  15.957
30 회차
loss :  16.118
31 회차
loss :  16.077
32 회차
loss :  16.037
33 회차
loss :  16.065
34 회차
loss :  16.118
35 회차
loss :  15.982
36 회차
loss :  15.81
37 회차
loss :  15.957
38 회차
loss :  16.118
39 회차
loss :  16.118
40 회차
loss :  16.118
41 회차
loss :  15.957
42 회차
loss :  16.118
43 회차
loss :  16.118
44 회차
loss :  16.118
45 회차
loss :  16.118
46 회차
loss :  15.957
47 회차
loss :  16.118
48 회차
loss :  15.957
49 회차


In [ ]:
y = np.array([[1,2,3],[4,5,6]])
x = np.random.rand(2)
print(x)
print(y)
print(y - x)

In [16]:
x = [1,2,3]
x.reverse()
x

[3, 2, 1]